## TSMixer: An All-MLP Architecture for Time Series Forecasting

##### Documentation:
https://unit8co.github.io/darts/generated_api/darts.models.forecasting.tsmixer_model.html

##### Paper:
https://arxiv.org/abs/2303.06053

### Important Terms
<b>Past covariates:</b> Time series whose past values are known at prediction time. Those series often contain values that have to be observed to be known.

<b>Future covariates:</b> Time series whose future values are already known at prediction time for the span of the forecast horizon. These can for instance represent known future holidays, or weather forecasts.

<b>Static covariates:</b> Constant over time (e.g., district name).

### Used Methods

Cells with bold fonts indicate the decided methods for the last submission.

| Segmentation  | Models             | Time       | Aggregation | Training              |
|---------------|--------------------|------------|-------------|-----------------------|
| <b>City</b>   | <b>TSMixerModel</b>| 1h         | <b>Mean</b> | <b>Train all once</b> |
| District      | TideModel          | <b>6h</b>  | <b>Std</b>  | Train per il          |
| TS Clustering | RNN (LSTM and GRU) | <b>8h</b>  | Min         | Train per ilce        |
| PCA           | DLinearModel       | <b>12h</b> | Max         |                       |
|               | NLinearModel       | <b>24h</b> | Skew        |                       |
|               |                    |            | Kurt        |                       |

In [ ]:
!pip install -qq darts # See https://unit8co.github.io/darts/

In [ ]:
from pathlib import Path

from tqdm import tqdm
from datetime import date
from itertools import product

import numpy as np
import pandas as pd
import polars as pl

import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler

import darts
from darts import TimeSeries
from darts.models import TiDEModel, TSMixerModel, RNNModel, NaiveEnsembleModel
from darts.dataprocessing.transformers import Scaler
from darts.utils.missing_values import fill_missing_values
from darts.utils.statistics import plot_acf, plot_pacf, check_seasonality

import torch

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Aggregation rules
weather_schema = {
    "t_2m:C": ["mean", "std"],
    "effective_cloud_cover:p": ["mean", "std"],
    "global_rad:W": ["mean", "std"],
    "relative_humidity_2m:p": ["mean", "std"],
    "wind_dir_10m:d": ["mean", "std"],
    "wind_speed_10m:ms": ["mean", "std"],
    "prob_precip_1h:p": ["mean", "std"],
    "t_apparent:C": ["mean", "std"],
}

# Encoder for future covariates
encoders = {
    'cyclic': {'future': ['dayofyear']},
    'datetime_attribute': {'future': ['dayofweek']},
    'position': {'past': ['relative'], 'future': ['relative']},
    'transformer': Scaler(StandardScaler()),
}

# Directory path
root = Path("/kaggle/input/gdz-elektrik-datathon-2024")

## Helper Functions

In [ ]:
def get_schema_output(schema:dict) -> list[str]:
    '''
    Extract output column names based on aggregation schema
    
    Parameters
    ----------
    schema: dict
        A dictionary where keys are column names and values are lists of aggregation functions to apply to each column.
    
    Returns
    -------
    list:
        A list of formatted column names based on the applied aggregations.
    '''
    col_names = []
    
    for col, agg_list in schema.items():
        col_names.extend([f"{agg}_{col}" for agg in agg_list])
        
    return col_names

In [ ]:
def aggregate(schema:dict) -> list[pl.Expr]:
    '''
    Generate aggregation expressions for polars
    
    Parameters
    ----------
    schema: dict
        A dictionary where keys are column names and values are lists of aggregation functions to apply to each column.
    
    Returns
    -------
    list:
        A list of dynamically generated polars expressions.
    '''
    expr_list = []
    
    for col, agg_list in schema.items():
        for agg in agg_list:
            expr = f'[pl.col("{col}").{agg}().alias("{agg}_{col}")]'
            expr_list = expr_list + eval(expr)
    
    return expr_list

In [ ]:
def get_data(train_path:str, test_path:str) -> pl.DataFrame:
    '''
    Read and preprocess data
    
    Parameters
    ----------
    train_path: str
        Training file path.
    
    test_path: str
        Test file path.
    
    Returns
    -------
    pl.DataFrame:
        Concatenated polars dataframe that missing dates filled with 0.
    '''
    return (
        pl.concat([
            pl.read_csv(
                train_path, 
                try_parse_dates=True,
            ),
            pl.read_csv(
                test_path, 
                try_parse_dates=True,
            ).with_columns(
                pl.lit(0).alias("bildirimsiz_sum"),
            )
            .select("tarih", "ilce", "bildirimsiz_sum", "bildirimli_sum"),
        ], how="vertical_relaxed")
        .pivot(
            index="tarih",
            columns="ilce",
            values=["bildirimsiz_sum", "bildirimli_sum"],
        )
        .sort("tarih")
        .upsample("tarih", every="1d")
        .fill_null(0)
    )

In [ ]:
def get_subm(path:str) -> pd.DataFrame:
    '''
    Read submission data
    
    Parameters
    ----------
    path: str
        Submission file path.
    
    Returns
    -------
    pd.DataFrame:
        Submission table.
    '''
    return (
        pd.read_csv(path)
        .set_index("unique_id")
    )

In [ ]:
def get_weather(path:str, schema:dict, every:str="24h", level:str="ilce") -> pl.DataFrame:
    '''
    Process weather data
    
    Parameters
    ----------
    path: str
        Weather file path.
    
    schema: dict
        A dictionary where keys are column names and values are lists of aggregation functions to apply to each column.
    
    every: str
        Time frequency downsampling rule. It can take values between 1h...24h.
        
    level:str:
        Region level to aggregate. It can take values in ["il", "ilce"].
    
    Returns
    -------
    pl.DataFrame:
        Weather table.
    '''
    return (
        pl.read_csv(path, try_parse_dates=True)
        .rename({"name": "ilce"})
        .with_columns(
            il = pl.col("ilce").str.split(by="-").list[0],
        )
        .sort("date", "il", "ilce")
        .group_by_dynamic(
            "date",
            every=every,
            group_by=level,
        )
        .agg(aggregate(schema))
        .with_columns(
            hour = pl.col("date").dt.hour(),
            date = pl.col("date").dt.date(),
        )
        .pivot(
            index=["date"],
            columns=["hour", level],
            values=get_schema_output(schema),
        )
        .drop("il", "ilce")
    )

In [ ]:
def feature_eng(df_data:pl.DataFrame, df_weather:pl.DataFrame) -> pd.DataFrame:
    '''
    Feature engineering
    
    Parameters
    ----------
    df_data: pl.DataFrame
        Training table.
        
    df_weather: pl.DataFrame
        Weather table.
    
    Returns
    -------
    pd.DataFrame:
        Final version of training table.
    '''
    return (
        df_data
        .join(df_weather, how="left", left_on="tarih", right_on="date")
        .to_pandas()
        .set_index("tarih")
    )

In [ ]:
def to_timeseries(df_data:pd.DataFrame) -> darts.TimeSeries:
    '''
    Convert DataFrame to TimeSeries
    
    Parameters
    ----------
    df_data: pd.DataFrame
        Training table.
    
    Returns
    -------
    darts.TimeSeries:
        A multivariate time series.
    '''
    return TimeSeries.from_dataframe(
        df_data
    )

In [ ]:
def handle_pred(pred:darts.TimeSeries) -> pd.DataFrame:
    '''
    Convert TimeSeries forecasts to submission ready format
    
    Parameters
    ----------
    pred: darts.TimeSeries
        Deep learning model forecasts.
    
    Returns
    -------
    pd.DataFrame:
        Forecasts in submission ready format.
    '''
    pred = pred.pd_dataframe()
    
    pred.columns = [col.split("_")[-1] for col in pred.columns]
    
    pred = pred.reset_index()
    pred = pred.melt(id_vars="tarih", value_vars=pred.columns[1:])
    
    pred["unique_id"] = pred["tarih"].astype(str) + "-" + pred["variable"]
    
    pred = pred.drop(columns=["tarih", "variable"])
    pred = pred.set_index("unique_id")
    pred = pred.rename(columns={"value": "bildirimsiz_sum"})
    
    return pred

## Data Preparation

In [ ]:
df_weather = get_weather(
    path=root / "weather.csv", 
    schema=weather_schema, 
    every="24h", 
    level="il",
)

df_data = get_data(
    train_path=root / "train.csv", 
    test_path=root / "test.csv",
)

df_data = feature_eng(
    df_data=df_data, 
    df_weather=df_weather,
)

In [ ]:
'''
Outages in district A also affect outages in district B. 
That's why we estimate all districts at once.
'''

df_data.info(verbose=True)

## Data Visualization - District

In [ ]:
plt.figure(figsize=(16, 5))

segment = 'bildirimsiz_sum_ilce_izmir-aliaga'

series = to_timeseries(df_data)
series = series.slice(pd.Timestamp("20231201"), pd.Timestamp("20240201"))
series = series[segment]
series.plot()

plt.title(segment)
plt.show()

## Data Visualization - District & Weather Relationship

In [ ]:
plt.figure(figsize=(16, 5))

segment = 'bildirimsiz_sum_ilce_izmir-konak'

series = to_timeseries(df_data)
series = series.slice(pd.Timestamp("20231201"), pd.Timestamp("20240201"))
series = series[segment]
series.plot()

weather = 'mean_prob_precip_1h:p_{"hour","il"}_{0,"Izmir"}'

series = to_timeseries(df_data)
series = series.slice(pd.Timestamp("20231201"), pd.Timestamp("20240201"))
series = series[weather]
series.plot()

plt.title(segment)
plt.show()

## Data Visualization - District & District Relationship

In [ ]:
plt.figure(figsize=(16, 5))

segments = ['bildirimsiz_sum_ilce_izmir-konak','bildirimsiz_sum_ilce_izmir-bornova']

series = to_timeseries(df_data)
series = series.slice(pd.Timestamp("20231201"), pd.Timestamp("20240201"))
series = series[segments]
series.plot()

plt.title(segments)
plt.show()

## Time Series Analysis

### Partial Auto Correlation

In [ ]:
segment = "bildirimsiz_sum_ilce_manisa-akhisar"

series = to_timeseries(df_data)
series = fill_missing_values(series[segment])
series = series.drop_after(pd.Timestamp("20240201"))

plot_pacf(series, m=7, alpha=0.05, max_lag=30)
plt.show()

### Check Seasonality

In [ ]:
segment = "bildirimsiz_sum_ilce_izmir-konak"

series = to_timeseries(df_data)
series = fill_missing_values(series[segment])
series = series.drop_after(pd.Timestamp("20240201"))

print("Seasonality(7): ", check_seasonality(series, m=7, max_lag=365))
print("Seasonality(7): ", check_seasonality(series, m=30, max_lag=365))

## Training Step

In [ ]:
# Convert DataFrame to TimeSeries
series = to_timeseries(df_data)

# Fill missing values in the time series
series = fill_missing_values(series)

# Extract target columns (columns containing "bildirimsiz_sum") and future covariate columns
is_target = series.columns.str.contains("bildirimsiz_sum")

target_cols = list(series.columns[is_target])
future_covariate_cols = list(series.columns[~is_target])

# Extract future covariates from the series
future_covariates = series[future_covariate_cols]

# Split the series into train, validation, and test sets based on time
train_series, test_series = series.split_before(pd.Timestamp("20240201"))
train_series, val_series = train_series.split_before(pd.Timestamp("20240104"))

# Extract target variables from train and validation
train_target = train_series[target_cols]
val_target = val_series[target_cols]

# Define the TSMixerModel with specific parameters
model = TSMixerModel(
    input_chunk_length=12,
    output_chunk_length=4,
    add_encoders=encoders,
    num_blocks=4,
    optimizer_kwargs={"lr": 3e-4},
    loss_fn=torch.nn.L1Loss(),
    n_epochs=100,
    batch_size=32,
    random_state=42,
    # pl_trainer_kwargs={"accelerator": "gpu"}, # Uncomment for GPU acceleration
)

# Fit the model to the training data
model.fit(
    series=train_target,
    future_covariates=future_covariates,
    val_series=val_target,
    val_future_covariates=future_covariates,
)

## Validation Step

In [ ]:
prediction = model.predict(len(val_target))

segment = 'bildirimsiz_sum_ilce_izmir-cesme'

plt.figure(figsize=(16, 5))

train_target[segment][-30:].plot(label="train")
val_target[segment].plot(label="val")
prediction[segment].plot(label="pred")

plt.show()

## Prediction Step

In [ ]:
prediction = model.predict(29, val_target)

segment = 'bildirimsiz_sum_ilce_izmir-cesme'

plt.figure(figsize=(16, 5))

val_target[segment].plot(label="val")
prediction[segment].plot(label="pred")

plt.show()

## Submission

In [ ]:
# Read the sample submission DataFrame and initialize the "bildirimsiz_sum" column to 0
df_subm = get_subm(root / "sample_submission.csv")
df_subm["bildirimsiz_sum"] = 0

# Define parameters for experimentation
seeds = [24, 42]
levels = ["il"]
hours = ["6h", "8h", "12h", "24h"]

# Iterate through combinations of seed, level, and hour using tqdm for progress tracking
for seed, level, hour in tqdm(product(seeds, levels, hours)):

    # Obtain weather data and preprocess it
    df_weather = get_weather(root / "weather.csv", weather_schema, hour, level)
    
    # Obtain and preprocess data for training and testing
    df_data = get_data(root / "train.csv", root / "test.csv")
    df_data = feature_eng(df_data, df_weather)
    
    # Convert DataFrame to TimeSeries and fill missing values
    series = to_timeseries(df_data)
    series = fill_missing_values(series)
    
    # Extract target columns and future covariate columns
    is_target = series.columns.str.contains("bildirimsiz_sum")
    
    target_cols = list(series.columns[is_target])
    future_covariate_cols = list(series.columns[~is_target])
    
    future_covariates = series[future_covariate_cols]

    # Split the time series into training and test sets based on a specific date
    train_series, test_series = series.split_before(pd.Timestamp("20240201"))

    # Extract target variables from the training set
    train_target = train_series[target_cols]
    
    # Define an ensemble model consisting of two TSMixer models
    model = NaiveEnsembleModel([
        TSMixerModel(
            input_chunk_length=12,
            output_chunk_length=4,
            n_epochs=100,
            batch_size=42,
            add_encoders=encoders,
            optimizer_kwargs={"lr": 0.00044837740437313645},
            loss_fn=torch.nn.L1Loss(),
            norm_type="LayerNorm",
            activation="ReLU",
            num_blocks=4,
            ff_size=54,
            hidden_size=63,
            random_state=seed,
            pl_trainer_kwargs={"accelerator": "gpu"}, # Uncomment for GPU acceleration
        ),
        TSMixerModel(
            input_chunk_length=12,
            output_chunk_length=4,
            n_epochs=100,
            batch_size=45,
            add_encoders=encoders,
            optimizer_kwargs={"lr": 0.00031599893639713627},
            loss_fn=torch.nn.L1Loss(),
            norm_type="TimeBatchNorm2d",
            activation="ReLU",
            num_blocks=4,
            ff_size=82,
            hidden_size=72,
            random_state=seed,
            pl_trainer_kwargs={"accelerator": "gpu"}, # Uncomment for GPU acceleration
        )
    ])
    
    # Fit the ensemble model to the training data
    model.fit(
        series=train_target,
        future_covariates=future_covariates,
    )

    # Generate predictions for the test set
    prediction = model.predict(29, verbose=False)
    prediction = handle_pred(prediction)
    
    # Update the "bildirimsiz_sum" column in the sample submission DataFrame with the predicted values
    df_subm["bildirimsiz_sum"] += prediction["bildirimsiz_sum"] / 8

In [ ]:
df_subm["bildirimsiz_sum"] = df_subm["bildirimsiz_sum"].round()

df_subm.to_csv("submission.csv")